In [1]:
import numpy as np
import torch as tc
from itertools import permutations


In [2]:
def merge_and_count(arr, temp_arr, left, mid, right):
    i = left    # 左子数组的起始索引
    j = mid + 1 # 右子数组的起始索引
    k = left    # 合并后的数组的起始索引
    inv_count = 0

    while i <= mid and j <= right:
        if arr[i] <= arr[j]:
            temp_arr[k] = arr[i]
            i += 1
        else:
            temp_arr[k] = arr[j]
            inv_count += (mid - i + 1)  # 计算逆序数
            j += 1
        k += 1

    # 遍历左子序列中剩下的
    while i <= mid:
        temp_arr[k] = arr[i]
        i += 1
        k += 1

    # 遍历右子序列中剩下的
    while j <= right:
        temp_arr[k] = arr[j]
        j += 1
        k += 1

    for i in range(left, right + 1):
        arr[i] = temp_arr[i]

    return inv_count

def merge_sort_and_count(arr, temp_arr, left, right):
    temp_arr = [0] * len(arr)
    inv_count = 0
    if left < right:
        mid = (left + right) // 2

        inv_count += merge_sort_and_count(arr, temp_arr, left, mid)
        inv_count += merge_sort_and_count(arr, temp_arr, mid + 1, right)
        inv_count += merge_and_count(arr, temp_arr, left, mid, right)

    return inv_count

# 示例用法
arr = [5, 1, 2, 3, 4]
temp_arr = [0] * len(arr)
result = merge_sort_and_count(arr, temp_arr, 0, len(arr) - 1)
print(f"逆序数是: {result}")

逆序数是: 4


In [42]:
def select_base(n:int, base_type:str):
    if base_type == 'Fourier':
        funcs = []
        for i in range(0, n):
            funcs.append(lambda r: tc.exp(1j*i*r) / pow(2 * tc.pi, 0.5))
    return funcs

def base_funcs(n:int, base_type:str, r_list:list):
    funcs = select_base(n, base_type)
    func_value = tc.zeros(size=(n, n), dtype=r_list[0].dtype)
    for i, func in enumerate(funcs):
        for j, r in enumerate(r_list):
            func_value[i, j] = func(r)
    return func_value

def contract(C_mps:list, phi_list:list):
    psi_ = 1
    for i in range(len(C_mps)):
        # print(C_mps[i].squeeze().shape)
        # print(phi_list[i].squeeze().shape)
        psi_ = psi_ * tc.einsum('i, i->', C_mps[i].squeeze(), phi_list[i].squeeze())
    return psi_

def test_func_symmetre(n:int, C_mps:list, base_type:str, r_list:list):
    func_value = base_funcs(n, base_type, r_list)
    # C_mps = list(10*tc.rand(size=(1, n, 1), dtype=r_list[0].dtype) for _ in range(n))
    psi = 0
    for perm in permutations(range(n), n):
        inv_count = merge_sort_and_count(list(perm), [0]*len(perm), 0, len(perm) - 1)
        phi_list = [func_value[:, i] for i in (perm)]
        psi = psi + contract(C_mps, phi_list) * (-1)**(inv_count)
    return psi

In [49]:
r = [10 * i * tc.ones([1,], dtype=tc.complex64) * tc.pi for i in range(4)]
C_mps = list(100*tc.rand(size=(1, 4, 1), dtype=r[0].dtype) for _ in range(4))
func_value = test_func_symmetre(n=4, C_mps=C_mps, base_type='Fourier', r_list=r)
print(func_value)
r_ = [r[1], r[0], r[2], r[3]]
func_value = test_func_symmetre(n=4, C_mps=C_mps, base_type='Fourier', r_list=r_)
print(func_value)


tensor(0.-8.j)
tensor(0.+8.j)
